Mandatory assignment 2 for the group:

Amitai Paz Iversen (S153660)
Thomas Rald Kaspersen (S151997)
Valdemar Hedegaard (S151940)
August Bjerg-Heise (S152153)

General assumptions:

Euclidian distance is fine (not haversian)
Rides are paid by the kilometer (5DKK)
Length of rides are assumed to be the manhattan distance between the unlock and the lock 

In [165]:
import numpy as np

price_per_m_DKK = 0.015

The code cell below contains the functionality of the scooter class. They focus on the "anywhere release" functionality, but must be handed in within a reasonable distance of a station. Shortly put, it does the following:

1. Initialization of the scooter class.
2. Scooters can be located anywhere globally.
3. Checks if the scooter is operational and within a 10-meter radius of the user before unlocking.
4. Stores the location before and after the ride to calculate the distance traveled.
5. Ensures the scooter is within a reasonable distance to a station before returning.
6. Assumes the scooter uses 1% of its battery for every 500 meters driven.



In [166]:
class Scooter:
    def __init__(self, scooter_id, position, status='working', station=None):
        self.scooter_id = scooter_id # Unique identifier for each scooter
        self.status = status # Status of the scooter (scooters are initially working)
        self.locked = True # Scooters are initially locked
        self.battery = 1 # Battery level of the scooter, with 1 equalling 100% (scooters are initially fully charged)
        self.station = station # Station where the scooter is currently located. The scooter is only located on a station when it is charging or being repaired
        self.latest_distance_travelled = 10 # Distance travelled by the scooter in the last ride
        if isinstance(position, tuple) and len(position) == 2 and all(isinstance(coord, (int, float)) for coord in position): # The position must be a tuple with two numeric values representing global coordinates
            self.position = position
        else:
            raise ValueError("Position must be a tuple with two numeric values representing global coordinates.")
    
    def __repr__(self):
        return f"Scooter_{self.scooter_id}"

    # Unlock the scooter for renting
    def unlock(self, User):
        
        # check if the user is close enough to the scooter (this distance is equivalent to approximately 10 meters)
        distance_to_user = np.linalg.norm(np.array(self.position) - np.array(User.position))
        if distance_to_user > 0.0001: 
            raise ValueError(f"User is too far from scooter {self.scooter_id}. Please get closer to unlock the scooter.") # if the user is too far, raise an error
        elif self.status != 'working':
            raise ValueError(f"Scooter {self.scooter_id} is not working. Please choose another scooter.") # check if the scooter is working
        else:
            self.locked = False # if the scooter is working and nearby, unlock it
        
        # stores the position of the latest unlock, in order to calculate the distance travelled
        self.latest_unlock_position = self.position 
        
        print(f"Scooter {self.scooter_id} is now unlocked and ready for use.")

    def lock(self, station, position): # Lock the scooter when returned
        self.position = position

        # check if the scooter is close enough to the station (this distance is equivalent to approximately 500 meters) before it is locked
        distance_to_station = np.linalg.norm(np.array(self.position) - np.array(station.position))
        if distance_to_station > 0.005: 
            raise ValueError(f"Station is too far from scooter {self.scooter_id}. Please get closer to lock the scooter.")
        else:  
            self.locked = True
    
        # calculate the distance travelled (useful to calculate the price of the ride and the battery used)
        self.latest_distance_travelled = 100000 * (np.abs(self.latest_unlock_position[0] - self.position[0]) + np.abs(self.latest_unlock_position[1] - self.position[1])) #store the manhattan distance between the latest unlock position and the current position in meters
        
        self.battery -= 0.01 * 500 * self.latest_distance_travelled # 1% of the battery is used for every 500 meters travelled

        # when the ride ends, there is a 1% chance that the scooter will break
        if np.random.rand() < 0.01:
            self.status = 'broken'
            print(f"Scooter {self.scooter_id} is now broken.")

        print(f"Scooter {self.scooter_id} is now locked.")
    

The cell below defines the station classes. There are two types of stations

1. Recharge stations: can recharge scooters
2. Repair stations: can recharge and repair scooters



In [167]:
class RechargeStation:
    def __init__(self, station_id, position):
        self.station_id = station_id
        self.position = position
        self.scooters = [] # scooters currently located at the station

    def __repr__(self):
        return f"RechargeStation_{self.station_id}"

    # retrieve scooter that is close (within 500m)
    def retrieve_scooter(self, scooter):
        scooter.position = self.position # set the scooter's position to the station's position
        self.scooters.append(scooter) # adds the scooter to the station's list of scooters

    def recharge(self, scooter):
        if scooter in self.scooters:
            scooter.battery = 1 # charge the scooter to 100%

    
# define a subclass of Station that can also repair scooters
class RepairStation(RechargeStation):
    # repair a broken scooter that is located at the station
    def repair_scooter(self, scooter):
        if scooter in self.scooters and scooter.status != 'working':
            scooter.status = 'working'
        print(f"Scooter {scooter.scooter_id} repaired at station {self.station_id}.")

    def __repr__(self):
        return f"RepairStation_{self.station_id}"


Then, the user is defined. Shortly put, the class can the following:

1. Defines a method (sorting algorithm) that finds the nearest scooter. Note that it is conscious choice to not use the sorting algorithm for slot management, since we chose to focus on the "release anywhere" function.
2. Defines a mthod for finding the nearest station
3. Account functionality for paying for trips
4. Methods for renting and returning scooters

In [176]:
class User:
    def __init__(self, membership_id, position):
        self.membership_id = membership_id
        self.position = position
        self.account = 0 # user's account balance for renting bikes. 

    def __repr__(self):
        return f"User_{self.membership_id}"

    # finds the nearest scooter that is working and charged above 20%
    def find_nearest_scooter(self, scooters):
        scooters = [scooter for scooter in scooters if scooter.status == 'working' and scooter.battery > 0.1] # remove scooters from the list that are not working or low battery
        nearest_scooter = min(scooters, key=lambda scooter: np.linalg.norm(np.array(scooter.position) - np.array(self.position)))  # find the scooter that has the shortest distance to the user
        print(f"Nearest scooter is {nearest_scooter.scooter_id}, which is located at {nearest_scooter.position}.") # helps the user locate the nearest scooter (since they need to be close to unlock it)
        return nearest_scooter
    
    def find_nearest_station(self, stations):
        nearest_station = min(stations, key=lambda station: np.linalg.norm(np.array(station.position) - np.array(self.position)))
        return nearest_station
    
    # method for depositing money into account
    def deposit(self, amount):
        self.account += amount
        print(f"User {self.membership_id} deposited {amount} into their account.")

    # method for renting a scooter
    def rent_scooter(self, scooter):
        scooter.unlock(self)
        print(f"Scooter {scooter.scooter_id} rented by user {self.membership_id}.")

    # method for returning a scooter
    def return_scooter(self, scooter):
        nearest_station = self.find_nearest_station(scooters)
        price = price_per_m_DKK * scooter.latest_distance_travelled # calculate the price of the ride
        self.account -= price # deduct the price of the ride from the user's account
        if self.account < 0:
            raise ValueError("Insufficient funds. Please deposit more money to your account.")
        scooter.lock(nearest_station, self.position)
        print(f"Scooter {scooter.scooter_id} returned by user {self.membership_id}. Total price is {price_per_m_DKK * scooter.latest_distance_travelled:.2f} DKK and total distance travelled was {scooter.latest_distance_travelled:.1f} meters.")

To test the flow, the subscirbers, stations and bikes are initiated. To keep the geographical scope managable, every instance is instantiated to be cover a 5*5 km square around copenhagen city centre.

In [177]:
# random seed for reproducibility
np.random.seed(69)

#Setting the geographic constraints for the simulation:
max_longitude = 12.6105
min_longitude = 12.5205
max_latitude = 55.7130
min_latitude = 55.6388

# Initialize the list of stations, stations and users
stations = []
scooters = []
users = []

# Define number of recharge and repair stations
num_recharge_stations = 90
num_repair_stations = 10
num_scooters = 5000
num_users = 10000

# Instantiate 90 recharge stations with readable names
for i in range(num_recharge_stations):
    station = RechargeStation(
        station_id= i+1,
        position=(np.random.uniform(min_longitude, max_longitude), np.random.uniform(min_latitude, max_latitude))
    )
    stations.append(station)

# Instantiate 10 repair stations with readable names
for i in range(num_repair_stations):
    station = RepairStation(
        station_id= i+1,
        position=(np.random.uniform(min_longitude, max_longitude), np.random.uniform(min_latitude, max_latitude))
    )
    stations.append(station)

# Instantiate 5000 scooters with readable names
for i in range(num_scooters):
    scooter = Scooter(
        scooter_id = i+1,
        position=(np.random.uniform(min_longitude, max_longitude), np.random.uniform(min_latitude, max_latitude)),
    )
    scooters.append(scooter)

# Instantiate 10000 users with readable names
for i in range(num_users):
    user = User(
        membership_id = i+1,
        position=(np.random.uniform(min_longitude, max_longitude), np.random.uniform(min_latitude, max_latitude))
    )
    users.append(user)

# Print the number of scooters, stations and users to check  
print(f"scooters: {len(scooters)}\nstations: {len(stations)}\nusers: {len(users)}")

scooters: 5000
stations: 100
users: 10000


In [178]:
# Select a user and a scooter for the demonstration
demo_user = users[0]  # Selecting the first user
demo_scooter = scooters[0]  # Selecting the first scooter

# User deposits money into their account
demo_user.deposit(100)  # Deposit 100 DKK

# User finds the nearest scooter
nearest_scooter = demo_user.find_nearest_scooter(scooters)

# if scooter is too far away, user moves to the scooter
if np.linalg.norm(np.array(nearest_scooter.position) - np.array(demo_user.position)) > 0.0001:
    demo_user.position = nearest_scooter.position

# User rents (unlocks) the nearest scooter
demo_user.rent_scooter(nearest_scooter)

# User moves to a new random position (simulating a ride)
print(f"scooter is located at {demo_scooter.position}")
new_position = (np.random.uniform(min_longitude, max_longitude), np.random.uniform(min_latitude, max_latitude))
demo_user.position = new_position
demo_scooter.position = new_position
print(f"scooter is now located at {demo_scooter.position}")

# User returns (locks) the scooter at the new position
demo_user.return_scooter(nearest_scooter)


User 1 deposited 100 into their account.
Nearest scooter is 1512, which is located at (12.590344866779303, 55.687068968777915).
Scooter 1512 is now unlocked and ready for use.
Scooter 1512 rented by user 1.
scooter is located at (12.59437977960228, 55.64582180142918)
scooter is now located at (12.592514543113687, 55.67124081527576)
Scooter 1512 is now locked.
Scooter 1512 returned by user 1. Total price is 27.00 DKK and total distance travelled was 1799.8 meters.
